Combining Bands Using GDAL

In [2]:
!gdal_translate -of VRT C:\Users\User\Documents\Sentinel-2\S2A_MSIL2A_20230905T105621_N0509_R094_T30UXB_20230905T151756\S2A_MSIL2A_20230905T105621_N0509_R094_T30UXB_20230905T151756.SAFE\GRANULE\L2A_T30UXB_A042848_20230905T105754\IMG_DATA\R10m\T30UXB_20230905T105621_B02_10m.jp2 B02.vrt
!gdal_translate -of VRT C:\Users\User\Documents\Sentinel-2\S2A_MSIL2A_20230905T105621_N0509_R094_T30UXB_20230905T151756\S2A_MSIL2A_20230905T105621_N0509_R094_T30UXB_20230905T151756.SAFE\GRANULE\L2A_T30UXB_A042848_20230905T105754\IMG_DATA\R10m\T30UXB_20230905T105621_B03_10m.jp2 B03.vrt
!gdal_translate -of VRT C:\Users\User\Documents\Sentinel-2\S2A_MSIL2A_20230905T105621_N0509_R094_T30UXB_20230905T151756\S2A_MSIL2A_20230905T105621_N0509_R094_T30UXB_20230905T151756.SAFE\GRANULE\L2A_T30UXB_A042848_20230905T105754\IMG_DATA\R10m\T30UXB_20230905T105621_B04_10m.jp2 B04.vrt


Input file size is 10980, 10980
Input file size is 10980, 10980
Input file size is 10980, 10980


In [3]:
!gdalbuildvrt -separate RGB.vrt B04.vrt B03.vrt B02.vrt


0...10...20...30...40...50...60...70...80...90...100 - done.


In [5]:
!gdal_translate -of GTiff RGB.vrt CompositeRGB.tif


Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.


In [4]:
!gdalbuildvrt -separate Composite.vrt RGB.vrt C:\Users\User\Documents\Sentinel-2\S2A_MSIL2A_20230905T105621_N0509_R094_T30UXB_20230905T151756\S2A_MSIL2A_20230905T105621_N0509_R094_T30UXB_20230905T151756.SAFE\GRANULE\L2A_T30UXB_A042848_20230905T105754\IMG_DATA\R10m\T30UXB_20230905T105621_TCI_10m.jp2 -vrtnodata "0 0 0 255"


0...10...20...30...40...50...60...70...80...90...100 - done.

Warning 1: RGB.vrt has 3 bands. Only the first one will be taken into account in the -separate case
Warning 1: C:\Users\User\Documents\Sentinel-2\S2A_MSIL2A_20230905T105621_N0509_R094_T30UXB_20230905T151756\S2A_MSIL2A_20230905T105621_N0509_R094_T30UXB_20230905T151756.SAFE\GRANULE\L2A_T30UXB_A042848_20230905T105754\IMG_DATA\R10m\T30UXB_20230905T105621_TCI_10m.jp2 has 3 bands. Only the first one will be taken into account in the -separate case


In [6]:
!gdal_translate -of GTiff Composite.vrt CompositeTCI_Layer.tif


Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.


Warning 1: CompositeTCI_Layer.tif: Unable to export GeoTIFF file with different datatypes per different bands. All bands should have the same types in TIFF.


Converting jp2 to Cloud Optimised GeoTiffs

In [14]:
import os
from osgeo import gdal

# Specify the input folder and output folder
input_folder = 'C:/Users/User/Documents/Sentinel-2/S2A_MSIL2A_20230905T105621_N0509_R094_T30UXB_20230905T151756/S2A_MSIL2A_20230905T105621_N0509_R094_T30UXB_20230905T151756.SAFE/GRANULE/L2A_T30UXB_A042848_20230905T105754/IMG_DATA/R10m'  # Replace with the path to your input folder
output_folder = 'COG files'  # This will create a new folder called "Processed files"
error_log_file = 'error_log.txt'  # Name of the error log file

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Initialize an error log file
with open(error_log_file, 'w') as error_log:
    error_log.write("Error Log:\n\n")

# Loop through all files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.jp2'):  # Adjust the file extension as needed
        input_filepath = os.path.join(input_folder, filename)
        output_filename = os.path.splitext(filename)[0] + '.tif'  # Change the extension to .tif
        output_filepath = os.path.join(output_folder, output_filename)

        try:
            # Open the input JP2 image
            input_image = gdal.Open(input_filepath)

            if input_image:
                # Create an output image in GeoTIFF format
                driver = gdal.GetDriverByName("GTiff")  # Specify the GeoTIFF driver
                output_image = driver.CreateCopy(output_filepath, input_image, options=["COMPRESS=DEFLATE", "TILED=YES"])

                # Close the input and output images
                input_image = None
                output_image = None

                print(f"Processed: {filename}")
            else:
                print(f"Failed to process: {filename}")
                # Log the error in the error log file
                with open(error_log_file, 'a') as error_log:
                    error_log.write(f"Failed to process: {filename}\n")

        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")
            # Log the error in the error log file
            with open(error_log_file, 'a') as error_log:
                error_log.write(f"Error processing {filename}: {str(e)}\n")

print("Processing completed.")


Processed: T30UXB_20230905T105621_AOT_10m.jp2
Processed: T30UXB_20230905T105621_B02_10m.jp2
Processed: T30UXB_20230905T105621_B03_10m.jp2
Processed: T30UXB_20230905T105621_B04_10m.jp2
Processed: T30UXB_20230905T105621_B08_10m.jp2
Processed: T30UXB_20230905T105621_TCI_10m.jp2
Processed: T30UXB_20230905T105621_WVP_10m.jp2
Processing completed.


In [15]:
import os
import subprocess

# Specify the folder containing the .tiff files
input_folder = 'C:/Users/User/COG files'  # Replace with the path to your input folder

# Loop through all files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.tif'):  # Process only .tif files
        input_filepath = os.path.join(input_folder, filename)
        
        # Specify the nodata value (0 in this case)
        nodata_value = 0
        
        # Create a temporary output file (with a different name)
        temp_output_filepath = os.path.join(input_folder, 'temp_' + filename)
        
        # Run gdal_translate to set nodata value and replace the original file
        gdal_translate_command = [
            'gdal_translate',
            '-a_nodata', str(nodata_value),
            input_filepath,
            temp_output_filepath
        ]
        
        # Execute the gdal_translate command
        subprocess.run(gdal_translate_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        
        # Replace the original file with the temp file
        os.replace(temp_output_filepath, input_filepath)
        
        print(f"Processed: {filename}")

print("Processing completed.")


Processed: T30UXB_20230905T105621_AOT_10m.tif
Processed: T30UXB_20230905T105621_B02_10m.tif
Processed: T30UXB_20230905T105621_B03_10m.tif
Processed: T30UXB_20230905T105621_B04_10m.tif
Processed: T30UXB_20230905T105621_B08_10m.tif
Processed: T30UXB_20230905T105621_TCI_10m.tif
Processed: T30UXB_20230905T105621_WVP_10m.tif
Processing completed.


In [31]:
import os
from PIL import Image
import rasterio
import numpy as np

# Specify the input folder containing the .tif files
input_folder = 'C:/Users/User/COG files'  # Replace with the path to your input folder

# Create an output folder inside the input folder path with the name "output_folder"
output_folder_name = 'output_folder'
output_folder = os.path.join(input_folder, output_folder_name)
os.makedirs(output_folder, exist_ok=True)

# Loop through all .tif files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.tif'):
        input_filepath = os.path.join(input_folder, filename)
        
        # Check if the filename contains "TCI"
        if "TCI" in filename:
            # Process the TCI image differently (original processing)
            try:
                with rasterio.open(input_filepath) as dataset:
                    # Extract the last 3 characters of the filename (excluding the extension)
                    filename_without_extension = os.path.splitext(filename)[0]
                    file_suffix = filename_without_extension[-7:]
                    
                    # Generate the names for the overview and thumbnail files
                    overview_filename = f'{file_suffix}_Overview.webp'
                    thumbnail_filename = f'{file_suffix}_Thumbnail.webp'
                    
                    # Read the first band as an array
                    original_image = dataset.read(1).astype(np.uint8)  # Read and convert to uint8
                    original_image = np.dstack((original_image, original_image, original_image))  # Convert to RGB
                    
                    # Create an overview image (lower resolution) using PIL
                    overview_image = Image.fromarray(original_image)
                    overview_image.thumbnail((500, 500))
                    overview_image.save(os.path.join(output_folder, overview_filename), 'WEBP', quality=95)
                    
                    # Create a thumbnail image (maximum 500px wide or high) using PIL
                    thumbnail_image = Image.fromarray(original_image)
                    thumbnail_image.thumbnail((500, 500))
                    thumbnail_image.save(os.path.join(output_folder, thumbnail_filename), 'WEBP', quality=95)
                    
                    print(f"Processed: {filename}")
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
        else:
            # Process other images differently (additional processing)
            try:
                with rasterio.open(input_filepath) as dataset:
                    # Extract the last 3 characters of the filename (excluding the extension)
                    filename_without_extension = os.path.splitext(filename)[0]
                    file_suffix = filename_without_extension[-7:]
                    
                    # Generate the names for the overview and thumbnail files
                    overview_filename = f'{file_suffix}_Overview.webp'
                    thumbnail_filename = f'{file_suffix}_Thumbnail.webp'
                    
                    # Read the first band as an array
                    original_image = dataset.read(1).astype(np.uint8)  # Read and convert to uint8
                    original_image = np.dstack((original_image, original_image, original_image))  # Convert to RGB
                    
                    # Create an overview image (lower resolution) using PIL
                    overview_image = Image.fromarray(original_image)
                    overview_image.thumbnail((500, 500))
                    overview_image.save(os.path.join(output_folder, overview_filename), 'WEBP', quality=95)
                    
                    # Create a thumbnail image (maximum 500px wide or high) using PIL
                    thumbnail_image = Image.fromarray(original_image)
                    thumbnail_image.thumbnail((500, 500))
                    thumbnail_image.save(os.path.join(output_folder, thumbnail_filename), 'WEBP', quality=95)
                    
                    print(f"Processed: {filename}")
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")

print("Processing completed.")


Processed: T30UXB_20230905T105621_AOT_10m.tif
Processed: T30UXB_20230905T105621_B02_10m.tif
Processed: T30UXB_20230905T105621_B03_10m.tif
Processed: T30UXB_20230905T105621_B04_10m.tif
Processed: T30UXB_20230905T105621_B08_10m.tif
Processed: T30UXB_20230905T105621_TCI_10m.tif
Processed: T30UXB_20230905T105621_WVP_10m.tif
Processing completed.


In [33]:
import os
import subprocess
import json

# Specify the input folder containing the TIFF files
input_folder = 'C:/Users/User/COG files'  # Replace with the path to your input folder

# Create a "metadata" folder inside the input folder path
metadata_folder_name = 'metadata'
metadata_folder = os.path.join(input_folder, metadata_folder_name)
os.makedirs(metadata_folder, exist_ok=True)

# Loop through all .tif files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.tif'):
        tif_file = os.path.join(input_folder, filename)

        # Define a dictionary to hold the STAC metadata for the current file
        stac_metadata = {
            "title": f"Sentinel-2A Image {filename}",  # Customize this title
            "description": f"Sentinel-2A 10m resolution image - Band {filename[-11:-8]}",  # Customize this description
            "instrument": "Multi-Spectral Instrument (MSI)",
            "satellite": "Sentinel-2A",
            "format": "COG",
            "resolution": "10 meters",
            "band": filename[-11:-8],  # Extract the band information from the filename
            # Add more metadata fields as needed
        }

        # Define the gdalinfo command as a list of arguments
        gdalinfo_command = ["gdalinfo", tif_file]

        # Run the gdalinfo command and capture the output
        try:
            result = subprocess.run(
                gdalinfo_command,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True,  # Ensure the output is treated as text
            )

            # Check for any errors
            if result.returncode == 0:
                # Parse the gdalinfo output
                gdalinfo_output = result.stdout
                stac_metadata["gdalinfo"] = gdalinfo_output.splitlines()

                # Save the STAC metadata as a JSON file in the "metadata" folder
                metadata_file_path = os.path.join(metadata_folder, f'metadata_{filename[-11:-8]}.json')
                with open(metadata_file_path, 'w') as metadata_file:
                    json.dump(stac_metadata, metadata_file, indent=4)

                print(f"STAC metadata saved to {metadata_file_path}")
            else:
                # Print the error message if the command failed
                print(f"Error processing {filename}: {result.stderr}")
        except FileNotFoundError:
            print("Error: gdalinfo command not found. Ensure GDAL is installed.")

print("Processing completed.")


STAC metadata saved to C:/Users/User/COG files\metadata\metadata_AOT.json
STAC metadata saved to C:/Users/User/COG files\metadata\metadata_B02.json
STAC metadata saved to C:/Users/User/COG files\metadata\metadata_B03.json
STAC metadata saved to C:/Users/User/COG files\metadata\metadata_B04.json
STAC metadata saved to C:/Users/User/COG files\metadata\metadata_B08.json
STAC metadata saved to C:/Users/User/COG files\metadata\metadata_TCI.json
STAC metadata saved to C:/Users/User/COG files\metadata\metadata_WVP.json
Processing completed.


In [36]:
import os
import subprocess
import json

# Define the path to the folder containing TIFF files
tif_folder = 'C:/Users/User/COG files'

# Create a function to get metadata for a TIFF file
def get_metadata(tif_file):
    # Define the gdalinfo command as a list of arguments
    gdalinfo_command = ["gdalinfo", tif_file]

    try:
        result = subprocess.run(
            gdalinfo_command,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,  # Ensure the output is treated as text
        )

        # Check for any errors
        if result.returncode == 0:
            # Parse the gdalinfo output
            gdalinfo_output = result.stdout

            # Extract relevant information from gdalinfo
            metadata = {
                "file_name": os.path.basename(tif_file),
                "gdalinfo": gdalinfo_output.splitlines(),
                # Add more metadata fields as needed
            }

            return metadata
        else:
            # Print the error message if the command failed
            print(f"Error processing {tif_file}: {result.stderr}")
            return None
    except FileNotFoundError:
        print("Error: gdalinfo command not found. Ensure GDAL is installed.")
        return None

# Create an "item_json" folder in the same path as the TIFF files
item_folder = os.path.join(tif_folder, 'item_json')
os.makedirs(item_folder, exist_ok=True)

# Iterate over TIFF files in the folder
for root, _, files in os.walk(tif_folder):
    for tif_file in files:
        if tif_file.endswith('.tif'):
            tif_file_path = os.path.join(root, tif_file)

            # Get metadata for the TIFF file
            metadata = get_metadata(tif_file_path)

            if metadata:
                # Create an item.json file for each TIFF file
                item_filename = os.path.splitext(tif_file)[0] + '_item.json'
                item_filepath = os.path.join(item_folder, item_filename)

                with open(item_filepath, 'w') as item_file:
                    json.dump(metadata, item_file, indent=4)

                print(f"Item JSON saved for {tif_file}: {item_filename}")


Item JSON saved for T30UXB_20230905T105621_AOT_10m.tif: T30UXB_20230905T105621_AOT_10m_item.json
Item JSON saved for T30UXB_20230905T105621_B02_10m.tif: T30UXB_20230905T105621_B02_10m_item.json
Item JSON saved for T30UXB_20230905T105621_B03_10m.tif: T30UXB_20230905T105621_B03_10m_item.json
Item JSON saved for T30UXB_20230905T105621_B04_10m.tif: T30UXB_20230905T105621_B04_10m_item.json
Item JSON saved for T30UXB_20230905T105621_B08_10m.tif: T30UXB_20230905T105621_B08_10m_item.json
Item JSON saved for T30UXB_20230905T105621_TCI_10m.tif: T30UXB_20230905T105621_TCI_10m_item.json
Item JSON saved for T30UXB_20230905T105621_WVP_10m.tif: T30UXB_20230905T105621_WVP_10m_item.json


In [ ]:
import json

# Define the fixed source information
source_info = {
    "Summary": {
        "Date": "2023-09-05T10:56:21.024Z",
        "Filename": "S2A_MSIL2A_20230905T105621_N0509_R094_T30UXB_20230905T151756.SAFE",
        "Identifier": "S2A_MSIL2A_20230905T105621_N0509_R094_T30UXB_20230905T151756",
        "Instrument": "MSI",
        "Satellite": "Sentinel-2",
        "Size": "529.93 MB"
    },
    "Product": {
        "Aot retrieval accuracy": 0.0,
        "Cloud cover percentage": 0.004322,
        "Cloud shadow percentage": 0.0,
        "Dark features percentage": 0.0,
        "Datastrip identifier": "S2A_OPER_MSI_L2A_DS_2APS_20230905T151756_S20230905T105754_N05.09",
        "Degraded ancillary data percentage": 0.0,
        "Degraded MSI data percentage": 0,
        # Add more product information as needed
    }
}

# Create a simple Collection JSON
collection = {
    "id": "sentinel-2-collection",
    "title": "Sentinel-2 Collection",
    "description": "Collection of Sentinel-2 data",
    "license": "CC-BY-4.0",
    "extent": {
        "spatial": {
            "bbox": [[-180, -90, 180, 90]]
        },
        "temporal": {
            "interval": [["2015-06-23T00:00:00Z", None]]
        }
    },
    "providers": [
        {
            "name": "European Space Agency",
            "roles": ["producer"],
            "url": "https://www.esa.int",
            "description": "Provider of Sentinel-2 data"
        }
    ],
    "catalogs": []
}

# Create a simple Catalog JSON
catalog = {
    "id": "sentinel-2-catalog",
    "title": "Sentinel-2 Catalog",
    "description": "Catalog of Sentinel-2 data",
    "license": "CC-BY-4.0",
    "extent": {
        "spatial": {
            "bbox": [[-180, -90, 180, 90]]
        },
        "temporal": {
            "interval": [["2015-06-23T00:00:00Z", None]]
        }
    },
    "providers": [
        {
            "name": "European Space Agency",
            "roles": ["producer"],
            "url": "https://www.esa.int",
            "description": "Provider of Sentinel-2 data"
        }
    ],
    "items": []
}

# Save the Collection and Catalog JSON to files
with open('collection.json', 'w') as collection_file:
    json.dump(collection, collection_file, indent=4)

with open('catalog.json', 'w') as catalog_file:
    json.dump(catalog, catalog_file, indent=4)

print("Collection and Catalog JSON files created.")


In [37]:
import json

collection_info = {
    "id": "sentinel-2A",
    "stac_version": "1.0.0",
    "title": "Sentinel-2A Imagery Collection",
    "description": "Collection of high-resolution satellite imagery of orbit (relative) 94 from the Sentinel-2A mission.",
    "license": "ISC",
    "extent": {
        "spatial": {
            "bbox": [
                [-0.9836426, 50.42629366061962, 0.017025008525814, 51.423790282941056]
            ]
        },
        "temporal": {
            "interval": [
                ["2023-09-05T10:56:21.024Z", "2023-09-05T10:56:21.024Z"]
            ]
        }
    },
    "summaries": {
        "license": ["ISC"],
        "providers": [
            {
                "name": "Sentinel-2A Mission",
                "roles": ["producer"],
                "url": "https://tinyurl.com/bdfxsdp6",
                "description": "Provider of Sentinel-2A satellite imagery."
            },
            {
                "name": "European Space Agency (ESA)",
                "roles": ["licensor"],
                "url": "https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-2",
                "description": "Licensor of Sentinel-2A imagery data."
            },
            {
                "name": "Open Cosmos",
                "roles": ["processor"],
                "url": "https://www.open-cosmos.com/",
                "description": "Processor of Sentinel-2A imagery data for demonstration purpose to examine candidates."
            }
        ]
    },
    "links": [
        {
            "rel": "self",
            "href": "https://example.com/collections/sentinel-2A"
        },
        {
            "rel": "parent",
            "href": "https://example.com/catalogs/catalog.json"
        },
        {
            "rel": "root",
            "href": "https://example.com/"
        }
    ]
}

# Save the Collection JSON to a file
with open('collection.json', 'w') as collection_file:
    json.dump(collection_info, collection_file, indent=4)

print("Collection JSON file created.")


Collection JSON file created.


In [42]:
import os
from google.cloud import storage

# Replace with your Google Cloud Storage bucket name
bucket_name = "sentinel-2"

# Local directory where your data is stored
local_data_directory = r"C:\Users\User\COG files\New folder"

# Path to your service account key JSON file
key_file_path = r"C:\Users\User\Downloads\civic-kayak-398904-3c5c2f482716.json"

def upload_to_gcs(bucket_name, source_file_path, destination_blob_name):
    """Uploads a file to Google Cloud Storage."""
    # Initialize the GCS client with your key file
    storage_client = storage.Client.from_service_account_json(key_file_path)

    # Get the bucket where you want to upload the file
    bucket = storage_client.bucket(bucket_name)

    # Create a blob (object) in the bucket
    blob = bucket.blob(destination_blob_name)

    # Upload the file to the blob
    blob.upload_from_filename(source_file_path)

    print(f"File {source_file_path} uploaded to {destination_blob_name} in bucket {bucket_name}.")

def upload_assets_to_gcs(bucket_name, local_data_directory):
    """Uploads TIFF files, overview.webp, thumbnail.webp, and item.json files to GCS."""
    # Iterate through the local data directory
    for root, _, files in os.walk(local_data_directory):
        for filename in files:
            # Determine the GCS destination path based on local directory structure
            relative_path = os.path.relpath(os.path.join(root, filename), local_data_directory)
            gcs_destination_blob_name = os.path.join("stac-data", relative_path.replace(os.sep, "/"))

            # Upload the file to GCS
            upload_to_gcs(bucket_name, os.path.join(root, filename), gcs_destination_blob_name)

if __name__ == "__main__":
    # Upload assets to GCS
    upload_assets_to_gcs(bucket_name, local_data_directory)


File C:\Users\User\COG files\New folder\item-2\asset-2.tif uploaded to stac-data\item-2/asset-2.tif in bucket sentinel-2.
File C:\Users\User\COG files\New folder\item-2\item.json uploaded to stac-data\item-2/item.json in bucket sentinel-2.
File C:\Users\User\COG files\New folder\item-2\metadata_B02.json uploaded to stac-data\item-2/metadata_B02.json in bucket sentinel-2.
File C:\Users\User\COG files\New folder\item-2\overview.webp uploaded to stac-data\item-2/overview.webp in bucket sentinel-2.
File C:\Users\User\COG files\New folder\item-2\thumbnail.webp uploaded to stac-data\item-2/thumbnail.webp in bucket sentinel-2.
File C:\Users\User\COG files\New folder\item-3\asset-3.tif uploaded to stac-data\item-3/asset-3.tif in bucket sentinel-2.
File C:\Users\User\COG files\New folder\item-3\item.json uploaded to stac-data\item-3/item.json in bucket sentinel-2.
File C:\Users\User\COG files\New folder\item-3\metadata_B03.json uploaded to stac-data\item-3/metadata_B03.json in bucket sentinel-2